## 导入包

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import netCDF4 as nc

from tqdm import tqdm
from netCDF4 import num2date

import matplotlib.pyplot as plt

from shapely.geometry import Point
# from shapely.validation import make_valid

import platform


发现mac系统下面，是有`from shapely.validation import make_valid`,但是在window上面不行

In [15]:
from shapely.geometry.base import geom_factory
from shapely.geos import lgeos


def make_valid(ob):
    if ob.is_valid:
        return ob
    else:
        return geom_factory(lgeos.GEOSMakeValid(ob._geom))

## 加载NC文件

In [5]:
nc_data = nc.Dataset("lftx.sfc.2021.nc")

## 处理NC数据

In [6]:
time_units = [temp_variable.units for temp_variable in nc_data.variables.values() if temp_variable.name == 'time'][0]
# [pd.to_datetime(str(num2date(temp_time,units=time_units))) for temp_time in np.array(nc_data.variables['time'])]
time_list = [num2date(temp_time, units=time_units).strftime('%Y-%m-%d') for temp_time in
             np.array(nc_data.variables['time'])]


In [7]:
lftx_data = np.array(nc_data.variables['lftx'])
lftx_data = lftx_data[np.where([i == '2021-02-13' for i in time_list])[0][0], :, :]

lon_data = np.array(nc_data.variables['lon'])
lat_data = np.array(nc_data.variables['lat'])

## 读取json数据

In [8]:
chinamap_data = gpd.read_file(filename="100000_中华人民共和国_full.json")
chinamap_data.head()

,adcode,name,childrenNum,level,parent,subFeatureIndex,adchar,geometry
0,110000,北京市,16.0,province,{'adcode': 100000},0.0,None,"MULTIPOLYGON (((117.34861 40.58114, 117.38988 ..."
1,120000,天津市,16.0,province,{'adcode': 100000},1.0,None,"MULTIPOLYGON (((117.76560 39.40053, 117.84691 ..."
2,130000,河北省,11.0,province,{'adcode': 100000},2.0,None,"MULTIPOLYGON (((117.46749 40.64974, 117.41267 ..."
3,140000,山西省,11.0,province,{'adcode': 100000},3.0,None,"MULTIPOLYGON (((110.37926 34.60061, 110.42484 ..."
4,150000,内蒙古自治区,12.0,province,{'adcode': 100000},4.0,None,"POLYGON ((97.17290 42.79526, 97.37123 42.45708..."


In [10]:
%matplotlib
fig, ax = plt.subplots(figsize=(6, 6))
chinamap_data.iloc[:35, :].plot(ax=ax, color='black')
plt.show()

Using matplotlib backend: Qt5Agg


In [11]:

#
# chinamap_data['geometry'].contains(Point(20, 40))

## 处理地图数据

In [12]:
Lon_data, Lat_data = np.meshgrid(lon_data, lat_data)

In [13]:
lftx_data_with_location = pd.DataFrame({'lon': Lon_data.flatten(),
                                        'lat': Lat_data.flatten(),
                                        'lftx': lftx_data.flatten()})
lftx_data_with_location.head()

,lon,lat,lftx
0,0.0,90.0,30.475
1,2.5,90.0,30.475
2,5.0,90.0,30.475
3,7.5,90.0,30.475
4,10.0,90.0,30.475


In [17]:
map_value = chinamap_data[['adcode', 'name', 'geometry']]
map_value['lftx_value'] = 0
map_value['num'] = 0


# map_value.head()

def trans(lon):
    """

    :param lon:
    :return:
    """
    if lon <= 180:
        return lon
    else:
        return lon - 360


# 这个代码非常重要
# https://stackoverflow.com/questions/63955752/topologicalerror-the-operation-geosintersection-r-could-not-be-performed

map_value['geometry'] = map_value.buffer(0)



# map_value['geometry'] = map_value['geometry'].apply(lambda x: make_valid(x))
map_value.loc[~map_value.is_valid]

<ipython-input-17-4acb89e82aa5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_value['lftx_value'] = 0
<ipython-input-17-4acb89e82aa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_value['num'] = 0
<ipython-input-17-4acb89e82aa5>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,adcode,name,geometry,lftx_value,num


In [18]:

for index in tqdm(range(lftx_data_with_location.shape[0])):
    temp_df = lftx_data_with_location.iloc[index, :]

    temp_mask = map_value['geometry'].contains(Point(trans(temp_df.lon),
                                                     temp_df.lat))
    map_value['lftx_value'] = map_value['lftx_value'] + temp_mask * temp_df.lftx

    map_value['num'] = map_value['num'] + temp_mask * 1

  0%|          | 0/10512 [00:00<?, ?it/s]<ipython-input-18-ff3ae4a514ba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_value['lftx_value'] = map_value['lftx_value'] + temp_mask * temp_df.lftx
<ipython-input-18-ff3ae4a514ba>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_value['num'] = map_value['num'] + temp_mask * 1
100%|██████████| 10512/10512 [00:12<00:00, 841.82it/s]


In [19]:
map_value['mean_lftx'] = map_value['lftx_value'] / map_value['num']  #(map_value['num'] + 0.00001)

map_value.loc[pd.isna(map_value['mean_lftx']), 'mean_lftx'] = 0
map_value.head()

<ipython-input-19-03ee3867ac12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_value['mean_lftx'] = map_value['lftx_value'] / map_value['num']  #(map_value['num'] + 0.00001)
C:\Users\yuanz\anaconda3\envs\mynet\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,adcode,name,geometry,lftx_value,num,mean_lftx
0,110000,北京市,"POLYGON ((115.72253 39.54420, 115.72068 39.551...",0.000000,0,0.000000
1,120000,天津市,"POLYGON ((116.91253 39.11090, 116.90945 39.150...",5.825000,1,5.825000
2,130000,河北省,"MULTIPOLYGON (((115.72253 39.54420, 115.72253 ...",21.725000,3,7.241667
3,140000,山西省,"POLYGON ((110.37926 34.60061, 110.29549 34.610...",10.250000,2,5.125000
4,150000,内蒙古自治区,"POLYGON ((97.17290 42.79526, 97.28254 42.78208...",289.474996,20,14.473750


## 画图


In [21]:
platform.system()

'Windows'

In [24]:
# 检测系统

if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
    # plt.rcParams['font.sans-serif']=['SimHei']
else:
    pass
# Linux
# Windows


fig, ax = plt.subplots()

map_value.plot(ax=ax, column='mean_lftx', legend=True)

map_value['center_lon'] = map_value['geometry'].centroid.x
map_value['center_lat'] = map_value['geometry'].centroid.y

for index in range(map_value.shape[0]):
    # ax.scatter(map_value['center_lon'], map_value['center_lat'])
    temp_df = map_value.iloc[index, :]
    ax.text(x=temp_df.center_lon, y=temp_df.center_lat, s=str(temp_df['name']))

ax.set_title("demo 中国地图, 公众号：pypi", fontdict={"size": 20})
ax.autoscale()
plt.tight_layout()

'Darwin'